In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

# Display the version
print(tf.__version__)

# import and mount Drive
import os
from google.colab import drive
drive.mount('/content/gdrive')

2.13.0
Mounted at /content/gdrive


In [2]:
# get path to folder file is in
!pip install kora
from kora.drive import get_path
file_path = get_path('17rNCLBHSPszyoaacFSpsFGD5nlP1OBF7')
file_path = '/content/gdrive/' + str(file_path) + '/Fashion-MNIST'
print(file_path)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.0 MB/s eta 0:00:00
/content/gdrive/My Drive/Personal Projects/Pioneer Academics Research/code/Fashion-MNIST


In [3]:
# Download the data from tf, unless it's already here.
if not os.path.exists(file_path+'/data/clean'):
  os.mkdir(file_path+'/data/clean')
  (X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
  np.save(file_path+'/data/clean/X_train.npy',X_train)
  np.save(file_path+'/data/clean/y_train.npy',y_train)
  np.save(file_path+'/data/clean/X_test.npy',X_test)
  np.save(file_path+'/data/clean/y_test.npy',y_test)
else:
  X_train = np.load(file_path+'/data/clean/X_train.npy')
  y_train = np.load(file_path+'/data/clean/y_train.npy')
  X_test = np.load(file_path+'/data/clean/X_test.npy')
  y_test = np.load(file_path+'/data/clean/y_test.npy')

print("X_train shape", X_train.shape)
print("y_train shape", y_train.shape)
print("X_test shape", X_test.shape)
print("y_test shape", y_test.shape)

X_train shape (60000, 28, 28)
y_train shape (60000,)
X_test shape (10000, 28, 28)
y_test shape (10000,)


In [4]:
# Convert to float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Decrease pixel values
X_train = X_train / 255.0
X_test = X_test / 255.0

In [5]:
# Create FGSM Perturbed Data
def create_fgsm_perturbations(data_inputs, data_labels, epsilon = 0.1):
  input_tensor = tf.convert_to_tensor(data_inputs)
  label_tensor = tf.convert_to_tensor(data_labels)
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
  with tf.GradientTape() as tape:
    tape.watch(input_tensor)
    output_tensor = model(input_tensor)
    loss = loss_fn(label_tensor, output_tensor)

  gradient = tape.gradient(loss, input_tensor)
  perturbed_input_tensor = input_tensor + epsilon * tf.sign(gradient)
  return tf.cast(perturbed_input_tensor, dtype=input_tensor.dtype)

# Create FGSM perturbations if nonexisting, else load them
if not os.path.exists(file_path+'/data/FGSM_perturbed'):
  os.mkdir(file_path+'/data/FGSM_perturbed')

  perturbed_data = [None for _ in range(5)]
  for i in range(5):
    perturbed_data[i] = create_fgsm_perturbations(X_train[i * 10000:(i+1) * 10000], y_train[i * 10000:(i+1) * 10000])

  X_FGSM_train = np.concatenate(perturbed_data, axis=0)
  X_FGSM_test = create_fgsm_perturbations(X_test, y_test)

  np.save(file_path+'/data/FGSM_perturbed/X_FGSM_train.npy',X_FGSM_train)
  np.save(file_path+'/data/FGSM_perturbed/X_FGSM_test.npy',X_FGSM_test)
else:
  X_FGSM_train = np.load(file_path+'/data/FGSM_perturbed/X_FGSM_train.npy')
  X_FGSM_test = np.load(file_path+'/data/FGSM_perturbed/X_FGSM_test.npy')

print("X_FGSM_train shape", X_FGSM_train.shape)
print("y_train shape", y_train.shape)
print("X_FGSM_test shape", X_FGSM_test.shape)
print("y_test shape", y_test.shape)

X_FGSM_train shape (60000, 28, 28, 1)
y_train shape (60000,)
X_FGSM_test shape (10000, 28, 28, 1)
y_test shape (10000,)


In [6]:
# Clip perturbed training and test data to viable values
X_FGSM_train = np.clip(X_FGSM_train, a_min = 0.0, a_max = 1.0)
X_FGSM_test = np.clip(X_FGSM_test, a_min = 0.0, a_max = 1.0)

In [7]:
# Create PGD Perturbed Data
def create_pgd_perturbations(data_inputs, data_labels, epsilon = 0.01, n_iter = 10):
  input_tensor = tf.convert_to_tensor(data_inputs)
  label_tensor = tf.convert_to_tensor(data_labels)
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)

  perturbed_tensor = input_tensor
  for i in range(n_iter):
      with tf.GradientTape() as tape:
          tape.watch(perturbed_tensor)
          output_tensor = model(perturbed_tensor)
          loss = loss_fn(label_tensor, output_tensor)

      gradient = tape.gradient(loss, perturbed_tensor)

      perturbed_tensor = perturbed_tensor + gradient
      # Project perturbed_tensor onto the L-infinity ball around input_tensor
      perturbed_tensor = epsilon * tf.sign(
          perturbed_tensor - input_tensor) + input_tensor

  perturbed_tensor = tf.cast(perturbed_tensor, dtype=input_tensor.dtype)
  return perturbed_tensor

# Create PGD perturbations if nonexisting, else load them
if not os.path.exists(file_path+'/data/PGD_perturbed'):
  os.mkdir(file_path+'/data/PGD_perturbed')

  perturbed_data = [None for _ in range(5)]
  for i in range(5):
    perturbed_data[i] = create_fgsm_perturbations(X_train[i * 10000:(i+1) * 10000], y_train[i * 10000:(i+1) * 10000])

  X_PGD_train = np.concatenate(perturbed_data, axis=0)
  X_PGD_test = create_pgd_perturbations(X_test, y_test)

  np.save(file_path+'/data/PGD_perturbed/X_PGD_train.npy',X_PGD_train)
  np.save(file_path+'/data/PGD_perturbed/X_PGD_test.npy',X_PGD_test)
else:
  X_PGD_train = np.load(file_path+'/data/PGD_perturbed/X_PGD_train.npy')
  X_PGD_test = np.load(file_path+'/data/PGD_perturbed/X_PGD_test.npy')

print("X_PGD_train shape", X_PGD_train.shape)
print("y_train shape", y_train.shape)
print("X_PGD_test shape", X_PGD_test.shape)
print("y_test shape", y_test.shape)

X_PGD_train shape (60000, 28, 28, 1)
y_train shape (60000,)
X_PGD_test shape (10000, 28, 28, 1)
y_test shape (10000,)


In [8]:
# Clip perturbed training and test data to viable values
X_PGD_train = np.clip(X_PGD_train, a_min = 0.0, a_max = 1.0)
X_PGD_test = np.clip(X_PGD_test, a_min = 0.0, a_max = 1.0)

In [9]:
def evaluate(model):
  # Evaluate on Clean Test Data
  print("clean test data: ")
  model.evaluate(X_test, y_test)

  # Evaluate on FGSM-perturbed test data
  print("fgsm perturbed test data: ")
  model.evaluate(X_FGSM_test, y_test)

  # Evaluate on PGD-perturbed test data
  print("pgd perturbed test data: ")
  model.evaluate(X_PGD_test, y_test)

def evaluateDenoised(model, autoencoder):
  # Evaluate on (Denoised) Clean Test Data
  print("clean test data: ")
  model.evaluate(autoencoder.predict(X_test), y_test)

  # Evaluate on (Denoised) FGSM-perturbed test data
  print("fgsm perturbed test data: ")
  model.evaluate(autoencoder.predict(X_FGSM_test), y_test)

  # Evaluate on (Denoised) PGD-perturbed test data
  print("pgd perturbed test data: ")
  model.evaluate(autoencoder.predict(X_PGD_test), y_test)


# Load in Clean Model
print("clean model:\n")
model = tf.keras.models.load_model(file_path+'/models/clean.keras')
evaluate(model)

# Load in FGSM Model
print("\nfgsm model:\n")
fgsm_model = tf.keras.models.load_model(file_path+'/models/mixed_FGSM.keras')
evaluate(fgsm_model)

# Load in PGD Model
print("\npgd model:\n")
pgd_model = tf.keras.models.load_model(file_path+'/models/mixed_PGD.keras')
evaluate(pgd_model)

# Load in FGSM Denoising Autoencoder
print("\ndenoised fgsm noise, clean model:\n")
FGSM_denoising_autoencoder = tf.keras.models.load_model(file_path+'/models/FGSM_denoising_autoencoder.keras')
evaluateDenoised(model, FGSM_denoising_autoencoder)

# Load in PGD Denoising Autoencoder
print("\ndenoised pgd noise, clean model:\n")
PGD_denoising_autoencoder = tf.keras.models.load_model(file_path+'/models/PGD_denoising_autoencoder.keras')
evaluateDenoised(model, PGD_denoising_autoencoder)

def temperature_cross_entropy(gt, pred):
  loss = tf.nn.softmax_cross_entropy_with_logits(labels=gt, logits=pred/20)
  return loss

# Load in Distilled Model
print("\ndistilled model:\n")
distilled_model = tf.keras.models.load_model(file_path+'/models/distillation_student.keras', custom_objects={'temperature_cross_entropy': temperature_cross_entropy})
evaluate(distilled_model)

clean model:

clean test data: 
313/313 [==============================] - 13s 6ms/step - loss: 0.2981 - accuracy: 0.9259
fgsm perturbed test data: 
313/313 [==============================] - 1s 4ms/step - loss: 3.2335 - accuracy: 0.2201
pgd perturbed test data: 
313/313 [==============================] - 1s 4ms/step - loss: 0.5759 - accuracy: 0.8338

fgsm model:

clean test data: 
313/313 [==============================] - 1s 3ms/step - loss: 0.2973 - accuracy: 0.9259
fgsm perturbed test data: 
313/313 [==============================] - 1s 3ms/step - loss: 0.1580 - accuracy: 0.9775
pgd perturbed test data: 
313/313 [==============================] - 1s 3ms/step - loss: 0.3551 - accuracy: 0.9031

pgd model:

clean test data: 
313/313 [==============================] - 1s 3ms/step - loss: 0.3017 - accuracy: 0.9240
fgsm perturbed test data: 
313/313 [==============================] - 1s 3ms/step - loss: 0.1566 - accuracy: 0.9779
pgd perturbed test data: 
313/313 [========================